In [ ]:
import tensorflow as tf
import numpy as np

## CSV 前処理

In [ ]:
csv_filename = "log_20161110.csv"

+ csv ファイルは、列ヘッダなし・列数8列・全5902行
+ 各列は以下の通り：
    + ページ（パス、文字列）
    + ページビュー数（整数）
    + ページ別訪問数（整数）
    + 平均ページ滞在時間（整数）
    + 閲覧開始数（整数）
    + 直帰率（実数）
    + 離脱率（実数）
    + コンテンツサイズ（整数）

In [ ]:
csv_lines = open(csv_filename, "r").readlines()

In [ ]:
num_lines = len(csv_lines)
num_lines

In [ ]:
csv_lines[:6]

In [ ]:
train_idxs = np.random.choice(range(num_lines), num_lines*5//6, replace=False)

In [ ]:
train_data = np.array(csv_lines)[train_idxs]

In [ ]:
train_data

In [ ]:
csv_data = np.array(csv_lines)
np.random.shuffle(csv_data)
csv_data[:6]

In [ ]:
train_num = len(csv_lines) * 5 // 6
train_data = csv_data[:train_num]
test_data = csv_data[train_num:]

(len(train_data), len(test_data))

In [ ]:
with open("train.csv", "w") as f:
    for line in train_data:
        f.write(line)

In [ ]:
with open("test.csv", "w") as f:
    for line in test_data:
        f.write(line)

## グラフ構築

In [ ]:
denoms = {
    "pv": 4096,
    "visits": 4096,
    "duration": 128,
    "entrances": 1024,
    "bounce_rate": 1,
    # "exit_rate": 1,
    "contents_size": 4096
}

In [ ]:
tf_stack = tf.stack if hasattr(tf, "stack") else tf.pack
# ↑ TensorFlow 0.1x/1.0 の差分吸収

In [ ]:
def read_data(filename, batch_size=128, denoms=denoms):
    filename_queue = tf.train.string_input_producer([filename])

    # TextLineReader 生成（1行ずつ読み込む Reader）
    reader = tf.TextLineReader()
    key, value = reader.read(filename_queue)

    # CSVデコード
    # ※ 列は8列、文字列、整数、整数、整数、整数、実数、実数、整数。だが処理の都合上、
    # 　 最初以外は実数として扱う↓）
    page, pv, visits, duration, entrances, bounce_rate, exit_rate, contents_size = (
        tf.decode_csv(value, [['path'], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0]]))
    # x = tf_stack([pv, visits, duration, entrances, bounce_rate, contents_size])
    x = tf_stack([
        pv / denoms["pv"],
        visits / denoms["visits"],
        duration / denoms["duration"],
        entrances / denoms["entrances"],
        bounce_rate / denoms["bounce_rate"],
        contents_size / denoms["contents_size"]])
    # threshold 0.4 
    y = tf.floor(exit_rate + 0.6)

    x_, y_ = tf.train.batch(
          [x, y], batch_size=batch_size)
    return x_, tf.reshape(y_, [-1])

In [ ]:
x_train, y_train = read_data("train.csv")

In [ ]:
x_test, y_test = read_data("test.csv", batch_size=len(test_data))

In [ ]:
W1 = tf.Variable(tf.truncated_normal([6, 40], stddev=np.sqrt(2.0/6)))
b1 = tf.Variable(tf.zeros([40]))
W2 = tf.Variable(tf.truncated_normal([40, 6], stddev=np.sqrt(2.0/40)))
b2 = tf.Variable(tf.zeros([6]))
W3 = tf.Variable(tf.truncated_normal([6, 1], stddev=np.sqrt(2.0/5)))
b3 = tf.Variable(tf.zeros([1]))

In [ ]:
def inference(x, W1=W1, b1=b1, W2=W2, b2=b2, W3=W3, b3=b3):
    a1 = tf.matmul(x, W1) + b1
    z1 = tf.nn.relu(a1)
    a2 = tf.matmul(z1, W2) + b2
    z2 = tf.nn.relu(a2)
    a3 = tf.matmul(z2, W3) + b3
    # return tf.nn.sigmoid(tf.reshape(a3, [-1]))
    return tf.reshape(a3, [-1])

In [ ]:
y = inference(x_train)

In [ ]:
def tf_nn_sigmoid_cross_entropy_with_logits(logits, labels, name=None):
    if hasattr(tf, "pack"):
        return tf.nn.sigmoid_cross_entropy_with_logits(logits, labels, name)
    else:
        return tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits, name=name)

In [ ]:
loss = tf.reduce_mean(tf_nn_sigmoid_cross_entropy_with_logits(y, y_train))
# loss = tf.reduce_mean(tf.square(y_train - y))

In [ ]:
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
y_eval = tf.nn.sigmoid(inference(x_test))
# y_eval = inference(x_test)

In [ ]:
# Calculate Confusion Matrix.
tf_cm_idxs = tf.transpose(tf_stack([
        tf.cast(tf.round(y_eval) , tf.int64),
        tf.cast(y_test, tf.int64)
    ]))
tf_cm_sparse = tf.SparseTensor(tf_cm_idxs, tf.ones_like(y_eval, dtype=tf.float32), [2, 2])
tf_cm = tf.sparse_add(tf.zeros([2, 2], dtype=tf.float32), tf_cm_sparse)

In [ ]:
def calc_metrices(cm):
    tn, fn, fp, tp = np.reshape(cm, [-1])
    acc = (tp+tn)/(tp+fp+fn+tn)
    prc = 0.0 if tp == fp == 0 else tp/(tp+fp)
    rec = 0.0 if tp == fn == 0 else tp/(tp+fn)
    f = 0.0 if prc == rec == 0 else 2*prc*rec / (prc+rec)
    return (acc, prc, rec, f)

In [ ]:
def tf_initialize_all_variables():
    if hasattr(tf, "global_variables_initializer"):
        return tf.global_variables_initializer()
    else:
        return tf.initialize_all_variables()

In [ ]:
init = tf_initialize_all_variables()

## 学習

In [ ]:
with tf.Session() as sess:
    sess.run(init)

    # 入力 enqueue スレッド開始
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    try:
        # ミニバッチ処理ループ
        cnt = 0
        while not coord.should_stop() and cnt < 3000:
            _, loss_ = sess.run([train_step, loss])
            if cnt % 100 == 0:
                print("step: %d" % cnt)
                print("\tloss: {}".format(loss_))
                # y_test_, y_eval_ = sess.run([y_test, y_eval])
                # print(np.mean(y_test_ == np.round(y_eval_)))
                cm_ = sess.run(tf_cm)
                acc_, prc_, rec_, f_ = calc_metrices(cm_)
                print("\tAccuracy: {}".format(acc_))
                print("\tPrecision: {}".format(prc_))
                print("\tRecall: {}".format(rec_))
                print("\tF-measure: {}".format(f_))
            cnt += 1
            # break
        W1_, b1_, W2_, b2_, W3_, b3_ = sess.run([W1, b1, W2, b2, W3, b3])
        x_test_, y_test_, y_eval_, cm_ = sess.run([x_test, y_test, y_eval, tf_cm])
    finally:
        coord.request_stop()
        coord.join(threads)

In [ ]:
loss_

In [ ]:
cm_

In [ ]:
calc_metrices(cm_)

## 結果確認

In [ ]:
x_test_

In [ ]:
y_test_

In [ ]:
# (y_eval_, np.round(y_eval_))
np.round(y_eval_)

In [ ]:
(W1_, b1_)

In [ ]:
(W2_, b2_)

In [ ]:
(W3_, b3_)

In [ ]:
(W1_.shape, W2_.shape, W3_.shape)

In [ ]:
W1_.dot(W2_.dot(W3_))

## おまけ1：推測

In [ ]:
def np_relu(x):
    return np.maximum(x, 0.0)

In [ ]:
def np_sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
np_denoms = np.array([denoms[k] for k in [
    "pv", "visits", "duration", "entrances", "bounce_rate", "contents_size"]], dtype="float32")

In [ ]:
def predict(x, W1_=W1_, b1_=b1_, W2_=W2_, b2_=b2_, W3_=W3_, b3_=b3_):
    x_ = np.reshape(x, [-1, 6]) / np_denoms
    a1 = np.dot(x_, W1_) + b1_
    z1 = np_relu(a1)
    a2 = np.dot(z1, W2_) + b2_
    z2 = np_relu(a2)
    a3 = np.dot(z2, W3_) + b3_
    return np_sigmoid(a3)


In [ ]:
def create_x(csvline):
    _page, pv, visits, duration, entrances, bounce_rate, _exit_rate, contents_size = (
        csvline.split(','))
    return np.array([
        float(pv), float(visits), float(duration), 
        float(entrances), float(bounce_rate), float(contents_size)], dtype="float32")

In [ ]:
test_data[0]

In [ ]:
x_test__ = create_x(test_data[0])
x_test__

In [ ]:
predict(x_test__)

In [ ]:
y_predicted_ = predict(x_test_ * np_denoms)

In [ ]:
y_predicted_

## おまけ2：勾配計算を利用した重み算出

In [ ]:
def compute_gradients(x, y=None, W1_=W1_, b1_=b1_, W2_=W2_, b2_=b2_, W3_=W3_, b3_=b3_):
    with tf.Graph().as_default(), tf.Session() as sess:
        W1 = tf.Variable(W1_)
        b1 = tf.Variable(b1_)
        W2 = tf.Variable(W2_)
        b2 = tf.Variable(b2_)
        W3 = tf.Variable(W3_)
        b3 = tf.Variable(b3_)
        x_ = (np.reshape(x, [-1, 6]) / np.array([4096., 4096., 128., 1024., 1., 4096.])).astype("float32")
        y_ = np.zeros(x_.shape[0], dtype="float32") if y is None else np.reshape(y, [-1]).astype("float32")
        z3 = inference(x_, W1=W1, b1=b1, W2=W2, b2=b2, W3=W3, b3=b3)
        loss = tf_nn_sigmoid_cross_entropy_with_logits(z3, y_)
        grads = tf.train.GradientDescentOptimizer(0.5).compute_gradients(loss)
        init = tf_initialize_all_variables()
        sess.run(init)
        return sess.run(grads)

In [ ]:
WgW1, bgb1, WgW2, bgb2, WgW3, bgb3 = compute_gradients(x_test__)

In [ ]:
bgb1[0]
# ^- 2層目→1層目のbackpropの値

In [ ]:
W1_.dot(bgb1[0])
# ^- 1層目→入力のbackpropの値

In [ ]:
W1_.dot(bgb1[0]) * np_denoms